Download Price Paid Data and Load into DB

In [1]:
# Verifies that CSV file is up-to-date (else: D/L from Land Registry server)

from ppc_db import fetch_csv

url = 'http://prod.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-complete.csv'
dest = 'C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/'

file_name = fetch_csv(url,dest)

100%|██████████| 5.17G/5.17G [05:59<00:00, 14.4MB/s]


In [4]:
# Copy local file into Docker volume

from ppc_db import copy_file_to_docker_volume

mount_path = "/var/lib/postgresql/data"

copy_file_to_docker_volume(file_name+".csv",
                           container_name = "788303b91078",
                           local_path = "C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/",
                           mount_path = mount_path
                           )

Successfully copied C:/Users/jamie/OneDrive/Python/Py_24/Docker/Data/ to path /var/lib/postgresql/data (container 788303b91078)


In [15]:
# CREATE TABLE (SQL)

from ppc_db import sql_commitchanges

file_name = 'Oct24pricepaidcomplete'

db_args = {
    'host':'localhost',
    'database':'LandRegData',
    'user':'root',
    'password':'password'
 }

base = 'CREATE TABLE '+file_name+' '

fd = open('tbl_params.sql','r')
sqlFile = fd.read()
fd.close()

print(base+sqlFile)

#sql_commitchanges(query=base+sqlFile,**db_args)

# CREATE TABLE Oct24pricepaidcomplete (
#     tx_id VARCHAR(40) DEFAULT NULL,
#     price_gbp INT DEFAULT NULL,
#     date DATE DEFAULT NULL,
#     postcode VARCHAR(100) DEFAULT NULL,
#     prop_type VARCHAR(1) DEFAULT NULL,
#     new_build VARCHAR(1) DEFAULT NULL,
#     estate_type VARCHAR(1) DEFAULT NULL,
#     "1_building_name_no" VARCHAR(100) DEFAULT NULL,
#     "2_building_name_no" VARCHAR(100) DEFAULT NULL,
#     "3_street" VARCHAR(100) DEFAULT NULL,
#     "4_locality" VARCHAR(100) DEFAULT NULL,
#     "5_town_city" VARCHAR(100) DEFAULT NULL,
#     "6_district" VARCHAR(100) DEFAULT NULL,
#     "7_county" VARCHAR(100) DEFAULT NULL,
#     "COL 15" VARCHAR(1) DEFAULT NULL,
#     "COL 16" VARCHAR(1) DEFAULT NULL
# );

CREATE TABLE Oct24pricepaidcomplete (tx_id VARCHAR(40) DEFAULT NULL,
price_gbp INT DEFAULT NULL,
date DATE DEFAULT NULL,
postcode VARCHAR(100) DEFAULT NULL,
prop_type VARCHAR(1) DEFAULT NULL,
new_build VARCHAR(1) DEFAULT NULL,
estate_type VARCHAR(1) DEFAULT NULL,
"1_building_name_no" VARCHAR(100) DEFAULT NULL,
"2_building_name_no" VARCHAR(100) DEFAULT NULL,
"3_street" VARCHAR(100) DEFAULT NULL,
"4_locality" VARCHAR(100) DEFAULT NULL,
"5_town_city" VARCHAR(100) DEFAULT NULL,
"6_district" VARCHAR(100) DEFAULT NULL,
"7_county" VARCHAR(100) DEFAULT NULL,
"COL 15" VARCHAR(1) DEFAULT NULL,
"COL 16" VARCHAR(1) DEFAULT NULL)


In [10]:
# LOAD DATA (SQL)

mount_path = "/var/lib/postgresql/data"

start = "COPY "+file_name+" FROM '"+mount_path+"/"+file_name+".csv'"
end = """ WITH (FORMAT csv, HEADER false, DELIMITER ',', QUOTE '"', ESCAPE '"')"""

#sql_commitchanges(start+mid+end, **db_args)

In [ ]:
print(start+mid+end)

# #COPY Oct24pricepaidcomplete
# FROM '/var/lib/postgresql/data/Oct24pricepaidcomplete.csv'
# WITH (FORMAT csv, HEADER false, DELIMITER ',', QUOTE '"', ESCAPE '"');

LOAD DATA INFILE "/var/lib/postgresql/data/Oct24pricepaidcomplete.csv" INTO TABLE Oct24pricepaidcomplete COLUMNS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"' ESCAPED BY '"' LINES TERMINATED BY '\n'
